# 作業 : 實作英文-德文翻譯機器人
***
## [作業目標]

用 pytorch 實作一個英文-德文翻譯機器人

## [作業目標]

*   語言資料處理
*   使用 LSTM 建構 Encoder: EncoderLSTM
*   使用 LSTM 建構 Decoder: DecoderLSTM
*   搭建 Sequence to Sequence 模型: Seq2Seq
*   撰寫訓練函式
*   撰寫測試函式

## [問題]

在 Colab 實際上執行完這個範例後，請改用 BiLSTM 來建構 Encoder 與 Decoder


## 安裝 spacy

We'll also make use of spaCy to tokenize our data. To install spaCy, follow the instructions here making sure to install both the English and German models with:

## 引用需要的模組

In [1]:
import os
import spacy
import torch
import random
import pandas as pd
from pprint import pprint

from torch import nn, optim
from torchtext.data import Field, BucketIterator
from torchtext.datasets import Multi30k
from torch.utils.tensorboard import SummaryWriter

## 下載英文預料

In [2]:
!mkdir ./data
!mkdir ./data/multi30k
!python -m spacy download en
!ls ./data/multi30k -al
spacy_english = spacy.load("en_core_web_sm")
!ls ./data/multi30k -al

mkdir: cannot create directory ‘./data’: File exists
mkdir: cannot create directory ‘./data/multi30k’: File exists
2021-02-03 12:19:03.986623: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Pleaseuse the
full pipeline package name 'en_core_web_sm' instead.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
total 8
drwxr-xr-x 2 root root 4096 Feb  3 11:29 .
drwxr-xr-x 3 root root 4096 Feb  3 11:29 ..
total 8
drwxr-xr-x 2 root root 4096 Feb  3 11:29 .
drwxr-xr-x 3 root root 4096 Feb  3 11:29 ..


## 下載德語語料

In [3]:
!python -m spacy download de
spacy_de = spacy.load("de_core_news_sm")
!ls ./data/multi30k -al

2021-02-03 12:19:14.823946: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
⚠ As of spaCy v3.0, shortcuts like 'de' are deprecated. Pleaseuse the
full pipeline package name 'de_core_news_sm' instead.
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
total 8
drwxr-xr-x 2 root root 4096 Feb  3 11:29 .
drwxr-xr-x 3 root root 4096 Feb  3 11:29 ..


In [4]:
def tokenize_de(text):
  return [token.text for token in spacy_de.tokenizer(text)]

def tokenize_english(text):
  return [token.text for token in spacy_english.tokenizer(text)]

### Sample Run ###

sample_text = "I love machine learning"
print(tokenize_english(sample_text))

german = Field(tokenize=tokenize_de, lower=True,
               init_token="<sos>", eos_token="<eos>")

english = Field(tokenize=tokenize_english, lower=True,
               init_token="<sos>", eos_token="<eos>")

train_data, valid_data, test_data = Multi30k.splits(exts = (".en", ".en"),
                                                    fields=(german, english))

german.build_vocab(train_data, max_size=10000, min_freq=3)
english.build_vocab(train_data, max_size=10000, min_freq=3)

print(f"Unique tokens in source (german) vocabulary: {len(german.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(english.vocab)}")


['I', 'love', 'machine', 'learning']
Unique tokens in source (german) vocabulary: 4587
Unique tokens in target (en) vocabulary: 4556


In [5]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

print(train_data[5].__dict__.keys())
pprint(train_data[5].__dict__.values())

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000
dict_keys(['src', 'trg'])
dict_values([['a', 'man', 'in', 'green', 'holds', 'a', 'guitar', 'while', 'the', 'other', 'man', 'observes', 'his', 'shirt', '.'], ['a', 'man', 'in', 'green', 'holds', 'a', 'guitar', 'while', 'the', 'other', 'man', 'observes', 'his', 'shirt', '.']])


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32

train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, valid_data, test_data), 
                                                                      batch_size = BATCH_SIZE, 
                                                                      sort_within_batch=True,
                                                                      sort_key=lambda x: len(x.src),
                                                                      device = device)

In [7]:
max_len_eng = []
max_len_ger = []
for counter, data in enumerate(train_data):
  max_len_ger.append(len(data.src))
  max_len_eng.append(len(data.trg))
  if counter < 10 :
    print("German - ",*data.src, " Length - ", len(data.src))
    print("English - ",*data.trg, " Length - ", len(data.trg))
    print()


print("Maximum Length of English sentence {} and German sentence {} in the dataset".format(max(max_len_eng),max(max_len_ger)))
print("Minimum Length of English sentence {} and German sentence {} in the dataset".format(min(max_len_eng),min(max_len_ger)))

German -  two young , white males are outside near many bushes .  Length -  11
English -  two young , white males are outside near many bushes .  Length -  11

German -  several men in hard hats are operating a giant pulley system .  Length -  12
English -  several men in hard hats are operating a giant pulley system .  Length -  12

German -  a little girl climbing into a wooden playhouse .  Length -  9
English -  a little girl climbing into a wooden playhouse .  Length -  9

German -  a man in a blue shirt is standing on a ladder cleaning a window .  Length -  15
English -  a man in a blue shirt is standing on a ladder cleaning a window .  Length -  15

German -  two men are at the stove preparing food .  Length -  9
English -  two men are at the stove preparing food .  Length -  9

German -  a man in green holds a guitar while the other man observes his shirt .  Length -  15
English -  a man in green holds a guitar while the other man observes his shirt .  Length -  15

German -  a 

In [8]:
count = 0
for data in train_iterator:
  if count < 1 :
    print("Shapes", data.src.shape, data.trg.shape)
    print()
    print("German - ",*data.src, " Length - ", len(data.src))
    print()
    print("English - ",*data.trg, " Length - ", len(data.trg))
    temp_ger = data.src
    temp_eng = data.trg
    count += 1

Shapes torch.Size([10, 32]) torch.Size([12, 32])

German -  tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([   4,    4,    4, 1187,    4,   16,   46,  472,    4,    4,    4,    4,
           4,    4,    4,   23, 1161,   21,    4,    4,  460,    4,   21,   19,
         290,    4, 2565,   16,    4,  190,    4,  110], device='cuda:0') tensor([  51,   55,   32,  153,  167,  109,  100,    6,   23,   23,   15, 1554,
         173,    9,   26,  100,    9,  109,    9,   38,  107,   15, 1626,   31,
         357,    9,   11,   49,   34,  125,   32,   62], device='cuda:0') tensor([  32,  134,   90,  160,   97,   30,  983,  169,  150,   34,   31,   63,
          32,  401,   35,  180,  456,   15,   10,   12,   17,   13,    9,    6,
         695,    6,    7,    8,  174,    6,   10, 3493], device='cuda:0') tensor([  10,   13,    8,    6,    0,  116, 2783,   11,  961, 2721,  275,   36,
          11,  126,  598,   

In [9]:
temp_eng_idx = (temp_eng).cpu().detach().numpy()
temp_ger_idx = (temp_ger).cpu().detach().numpy()

In [10]:
df_eng_idx = pd.DataFrame(data = temp_eng_idx, columns = [str("S_")+str(x + 1) for x in range(BATCH_SIZE)])
df_eng_idx.index.name = 'Time Steps'
df_eng_idx.index = df_eng_idx.index + 1 
# df_eng_idx.to_csv('/content/idx.csv')
df_eng_idx


,S_1,S_2,S_3,S_4,S_5,S_6,S_7,S_8,S_9,S_10,S_11,S_12,S_13,S_14,S_15,S_16,S_17,S_18,S_19,S_20,S_21,S_22,S_23,S_24,S_25,S_26,S_27,S_28,S_29,S_30,S_31,S_32
Time Steps,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
2,4,4,4,1192,4,16,48,480,4,4,4,4,4,4,4,24,163,21,4,4,457,4,21,19,295,4,2560,16,4,196,4,110
3,53,55,33,157,170,115,104,6,24,24,14,1506,122,9,26,104,42,115,9,38,112,14,1634,32,362,9,11,50,34,128,33,63
4,33,137,91,164,99,30,992,175,154,34,32,64,33,407,35,186,210,14,10,12,17,13,9,6,705,6,7,8,179,6,10,3482
5,10,13,8,6,0,119,2787,11,972,2723,280,36,11,129,601,74,9,10,56,19,617,25,10,43,241,21,104,4,336,4,3960,1531
6,78,44,4,4,51,8,8,208,4,3657,4,8,719,4,76,958,465,1021,69,37,6,217,218,12,6,502,165,1800,4,1356,6,199
7,4,665,308,990,7,4,4,42,2854,4,238,4,33,248,4,12,6,44,4,866,4,10,4,72,4,1195,0,363,1054,51,4,4
8,157,102,281,664,1598,84,578,23,929,224,138,166,41,243,959,330,4,201,859,1782,164,419,1165,19,398,1625,0,4351,47,435,575,580
9,5,515,5,5,5,5,5,186,5,5,5,5,5,5,5,5,25,5,5,5,5,5,5,5,5,5,5,3044,416,5,5,5


In [11]:
idx2word = {idx: word for idx, word in enumerate(english.vocab.itos)}
df_eng_word = pd.DataFrame(columns = [str("S_")+str(x+1) for x in range(BATCH_SIZE)])
df_eng_word = df_eng_idx.replace(idx2word)
# df_eng_word.to_csv('/content/Words.csv')
df_eng_word

,S_1,S_2,S_3,S_4,S_5,S_6,S_7,S_8,S_9,S_10,S_11,S_12,S_13,S_14,S_15,S_16,S_17,S_18,S_19,S_20,S_21,S_22,S_23,S_24,S_25,S_26,S_27,S_28,S_29,S_30,S_31,S_32
Time Steps,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>
2,a,a,a,fast,a,two,three,skateboarder,a,a,a,a,a,a,a,young,long,an,a,a,six,a,an,people,motorcycle,a,pops,two,a,kids,a,four
3,little,child,girl,bicycle,dirt,older,girls,in,young,young,woman,naked,blond,man,black,girls,-,older,man,group,dogs,woman,homeless,sitting,police,man,and,women,boy,play,girl,children
4,girl,plays,sits,race,bike,men,hammer,jeans,couple,boy,sitting,person,girl,putting,dog,performing,haired,woman,is,of,are,with,man,in,officer,in,the,on,jumps,in,is,keeping
5,is,with,on,in,<unk>,talking,nails,and,enjoys,excitedly,inside,standing,and,together,leaps,some,man,is,looking,people,competing,white,is,front,rides,an,girls,a,onto,a,curling,warm
6,riding,her,a,a,up,on,on,t,a,shoots,a,on,brunette,a,over,kind,works,checking,into,playing,in,sunglasses,reading,of,in,apron,taking,movie,a,blow,in,under
7,a,father,swing,foreign,the,a,a,-,snowball,a,train,a,girl,wooden,a,of,in,her,a,carnival,a,is,a,other,a,grilling,<unk>,set,inflatable,up,a,a
8,bicycle,'s,ride,country,terrain,sidewalk,roof,shirt,fight,basketball,car,rock,walking,chair,log,dance,a,watch,telescope,games,race,laughing,magazine,people,parade,corn,<unk>,reviewing,water,playground,competition,blanket
9,.,boots,.,.,.,.,.,performing,.,.,.,.,.,.,.,.,white,.,.,.,.,.,.,.,.,.,.,film,slide,.,.,.


## 用 LSTM 搭建的 Encoder 類別: EncoderLSTM



In [12]:
class EncoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, drop_rate):
    super(EncoderLSTM, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    #self.input_size = input_size

    # Output size of the word embedding NN
    #self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Regularization parameter
    self.dropout = nn.Dropout(drop_rate)
    self.tag = True

    # Shape --------------------> (5376, 300) [input size, embedding dims]
    self.embedding = nn.Embedding(input_size, embedding_size)
    
    # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = drop_rate)

  # Shape of x (26, 32) [Sequence_length, batch_size]
  def forward(self, x):

    # Shape -----------> (26, 32, 300) [Sequence_length , batch_size , embedding dims]
    embedding = self.dropout(self.embedding(x))
    
    # Shape --> outputs (26, 32, 1024) [Sequence_length , batch_size , hidden_size]
    # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size]
    outputs, (hidden_state, cell_state) = self.LSTM(embedding)

    return hidden_state, cell_state

input_size_encoder = len(german.vocab)
encoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
encoder_dropout = 0.5

encoder_lstm = EncoderLSTM(input_size_encoder, encoder_embedding_size,
                           hidden_size, num_layers, encoder_dropout).to(device)
print(encoder_lstm)

EncoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(4587, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
)


## 用 LSTM 搭建的 decoder 類別: DecoderLSTM


In [13]:
class DecoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, drop_rate, output_size):
    super(DecoderLSTM, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    #self.input_size = input_size

    # Output size of the word embedding NN
    #self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Size of the one hot vectors that will be the output to the encoder (English Vocab Size)
    self.output_size = output_size

    # Regularization parameter
    self.dropout = nn.Dropout(drop_rate)

    # Shape --------------------> (5376, 300) [input size, embedding dims]
    self.embedding = nn.Embedding(input_size, embedding_size)

    # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = drop_rate)

    # Shape -----------> (1024, 4556) [embedding dims, hidden size, num layers]
    self.fc = nn.Linear(hidden_size, output_size)

  # Shape of x (32) [batch_size]
  def forward(self, x, hidden_state, cell_state):

    # Shape of x (1, 32) [1, batch_size]
    x = x.unsqueeze(0)

    # Shape -----------> (1, 32, 300) [1, batch_size, embedding dims]
    embedding = self.dropout(self.embedding(x))

    # Shape --> outputs (1, 32, 1024) [1, batch_size , hidden_size]
    # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size] (passing encoder's hs, cs - context vectors)
    outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))

    # Shape --> predictions (1, 32, 4556) [ 1, batch_size , output_size]
    predictions = self.fc(outputs)

    # Shape --> predictions (32, 4556) [batch_size , output_size]
    predictions = predictions.squeeze(0)

    return predictions, hidden_state, cell_state

input_size_decoder = len(english.vocab)
decoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
decoder_dropout = 0.5
output_size = len(english.vocab)

decoder_lstm = DecoderLSTM(input_size_decoder, decoder_embedding_size,
                           hidden_size, num_layers, decoder_dropout, output_size).to(device)
print(decoder_lstm)

DecoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(4556, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  (fc): Linear(in_features=1024, out_features=4556, bias=True)
)


In [14]:
for batch in train_iterator:
  print(batch.src.shape)
  print(batch.trg.shape)
  break

x = batch.trg[1]
print(x)

torch.Size([12, 32])
torch.Size([16, 32])
tensor([ 16, 765, 202,   4,   4,   4,   4,   4,   4,   4,   4,  16,   4,   4,
          4,   4,  16,   7,   4,   9,   4, 110,   4,   4,   4,   4,   4,  53,
          4,   4,  19,   4], device='cuda:0')


# Sequence to Sequence 類別

In [21]:
class Seq2Seq(nn.Module):
  def __init__(self, Encoder_LSTM, Decoder_LSTM):
    super(Seq2Seq, self).__init__()
    self.Encoder_LSTM = Encoder_LSTM
    self.Decoder_LSTM = Decoder_LSTM

  def forward(self, source, target, tfr=0.5):
    # Shape - Source : (10, 32) [(Sentence length german + some padding), Number of Sentences]
    batch_size = source.shape[1]

    # Shape - Source : (14, 32) [(Sentence length English + some padding), Number of Sentences]
    target_len = target.shape[0]
    target_vocab_size = len(english.vocab)
    
    # Shape --> outputs (14, 32, 5766) 
    outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

    # Shape --> (hs, cs) (2, 32, 1024) ,(2, 32, 1024) [num_layers, batch_size size, hidden_size] (contains encoder's hs, cs - context vectors)
    hidden_state, cell_state = self.Encoder_LSTM(source)

    # Shape of x (32 elements)
    x = target[0] # Trigger token <SOS>

    for i in range(1, target_len):
      # Shape --> output (32, 5766) 
      output, hidden_state, cell_state = self.Decoder_LSTM(x, hidden_state, cell_state)
      outputs[i] = output
      best_guess = output.argmax(1) # 0th dimension is batch size, 1st dimension is word embedding
      x = target[i] if random.random() < tfr else best_guess # Either pass the next word correctly from the dataset or use the earlier predicted word

    # Shape --> outputs (14, 32, 5766) 
    return outputs


In [22]:
# Hyperparameters
learning_rate = 0.001
writer = SummaryWriter(f"runs/loss_plot")
step = 0

model = Seq2Seq(encoder_lstm, decoder_lstm).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

pad_idx = english.vocab.stoi["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [23]:
model

Seq2Seq(
  (Encoder_LSTM): EncoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(4587, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  )
  (Decoder_LSTM): DecoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(4556, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=1024, out_features=4556, bias=True)
  )
)

In [24]:
def translate_sentence(model, sentence, german, english, device, max_length=50):

    if type(sentence) == str:
        tokens = tokenize_de(sentence)
    else:
        tokens = [token.lower() for token in sentence]
    tokens.insert(0, german.init_token)
    tokens.append(german.eos_token)
    text_to_indices = [german.vocab.stoi[token] for token in tokens]
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        hidden, cell = model.Encoder_LSTM(sentence_tensor)

    outputs = [english.vocab.stoi["<sos>"]]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.Decoder_LSTM(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == english.vocab.stoi["<eos>"]:
            break

    translated_sentence = [english.vocab.itos[idx] for idx in outputs]
    return translated_sentence[1:]

# 用來評估模型的函式: bleu
def bleu(data, model, german, english, device):
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["src"]
        trg = vars(example)["trg"]

        prediction = translate_sentence(model, src, german, english, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)

def checkpoint_and_save(model, best_loss, epoch, optimizer, epoch_loss):
    print('saving')
    print()
    state = {'model': model,'best_loss': best_loss,'epoch': epoch,'rng_state': torch.get_rng_state(), 'optimizer': optimizer.state_dict(),}
    torch.save(state, './checkpoint-NMT')
    torch.save(model.state_dict(),'./checkpoint-NMT-SD')

In [25]:
!nvidia-smi

Wed Feb  3 12:20:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    32W /  70W |   2013MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [26]:
epoch_loss = 0.0
num_epochs = 30
best_loss = 999999
best_epoch = -1
sentence1 = "ein mann in einem blauen hemd steht auf einer leiter und putzt ein fenster"
ts1  = []

for epoch in range(num_epochs):
  print("Epoch - {} / {}".format(epoch+1, num_epochs))
  model.eval()
  translated_sentence1 = translate_sentence(model, sentence1, german, english, device, max_length=50)
  print(f"Translated example sentence 1: \n {translated_sentence1}")
  ts1.append(translated_sentence1)

  model.train(True)
  for batch_idx, batch in enumerate(train_iterator):
    input = batch.src.to(device)
    target = batch.trg.to(device)

    # Pass the input and target for model's forward method
    output = model(input, target)
    output = output[1:].reshape(-1, output.shape[2])
    target = target[1:].reshape(-1)

    # Clear the accumulating gradients
    optimizer.zero_grad()

    # Calculate the loss value for every epoch
    loss = criterion(output, target)

    # Calculate the gradients for weights & biases using back-propagation
    loss.backward()

    # Clip the gradient value is it exceeds > 1
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

    # Update the weights values using the gradients we calculated using bp 
    optimizer.step()
    step += 1
    epoch_loss += loss.item()
    writer.add_scalar("Training loss", loss, global_step=step)

  if epoch_loss < best_loss:
    best_loss = epoch_loss
    best_epoch = epoch
    checkpoint_and_save(model, best_loss, epoch, optimizer, epoch_loss) 
    if ((epoch - best_epoch) >= 10):
      print("no improvement in 10 epochs, break")
      break
  print("Epoch_Loss - {}".format(loss.item()))
  print()
  
print(epoch_loss / len(train_iterator))

# score = bleu(test_data[1:100], model, german, english, device)
# print(f"Bleu score {score*100:.2f}")

Epoch - 1 / 30
Translated example sentence 1: 
 ['a', 'man', 'in', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', '.', '.', '<eos>']
saving

Epoch_Loss - 3.3219828605651855

Epoch - 2 / 30
Translated example sentence 1: 
 ['<unk>', '<unk>', 'with', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '.', '<eos>']
Epoch_Loss - 3.0540013313293457

Epoch - 3 / 30
Translated example sentence 1: 
 ['<unk>', '<unk>', 'in', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '.', '<eos>']
Epoch_Loss - 1.9232869148254395

Epoch - 4 / 30
Translated example sentence 1: 
 ['<unk>', '<unk>', 'in', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<eos>']
Epoch_Loss - 1.7466254234313965

Epoch - 5 / 30
Translated example sentence 1: 
 ['<unk>', '<unk>', 'in', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<eos>']
Epoch_Loss - 1.6